In [ ]:
!pip install -U tensorflow

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import tensorflow as tf
import sys

In [ ]:
def monte_carlo_paths(S_0, time_to_expiry, sigma, drift, seed, n_sims, n_timesteps):
    """
    Create random paths of a underlying following a browian geometric motion
    
    input:
    
    S_0 = Spot at t_0
    time_to_experiy = end of the timeseries (last observed time)
    sigma = the volatiltiy (sigma in the geometric brownian motion)
    drift = drift of the process
    n_sims = number of paths to generate
    n_timesteps = numbers of aquidistant time steps 
    
    return:
    
    a (n_timesteps x n_sims x 1) matrix
    """
    if seed > 0:
            np.random.seed(seed)
    stdnorm_random_variates = np.random.randn(n_sims, n_timesteps)
    S = S_0
    dt = time_to_expiry / stdnorm_random_variates.shape[1]
    r = drift
    # See Advanced Monte Carlo methods for barrier and related exotic options by Emmanuel Gobet
    S_T = S * np.cumprod(np.exp((r-sigma**2/2)*dt+sigma*np.sqrt(dt)*stdnorm_random_variates), axis=1)
    return np.reshape(np.transpose(np.c_[np.ones(n_sims)*S_0, S_T]), (n_timesteps+1, n_sims, 1))

In [ ]:
S_0 = 100
K = 100
r = 0
vol = 0.2
T = 1/12
timesteps = 30
seed = 42
n_sims = 10000

paths_train = monte_carlo_paths(S_0,T,vol,r,seed,n_sims,timesteps)

In [ ]:
STOCK_GRAPH = paths_train
#print(STOCK_GRAPH.shape) # (31,50000,1) 31일간 시나리오를 50000개 갖고 있다.

STOCK_T = np.reshape(STOCK_GRAPH[-1,:,0],(-1,1))
print(STOCK_T.shape)
STOCK_dS = STOCK_GRAPH[1:,:,0] - STOCK_GRAPH[0:-1,:,0]
STOCK_dS = np.swapaxes(STOCK_dS,0,1)
#STOCK_dS = STOCK_dS.astype(dtype='float64')
STOCK_K = np.full((n_sims,1),100)
print(STOCK_K.shape,STOCK_dS.shape)
STOCK_GRAPH = tf.unstack(STOCK_GRAPH[:-1,:,:],axis=0)
STOCK_GRAPH = tf.stack(STOCK_GRAPH)
STOCK_GRAPH = np.swapaxes(STOCK_GRAPH,0,1)
#STOCK_GRAPH = np.reshape(STOCK_GRAPH,(30,10000))
#print(STOCK_GRAPH.shape)

(10000, 1)
(10000, 1) (10000, 30)


In [ ]:
batch_size =  1000
features = 1
K = 100
alpha = 0.50
epoch = 100

In [ ]:
def deep_lstm_model():
    
    input = tf.keras.layers.Input(shape=(30,1)) # (time_steps,feature)
    # cell_state,hidden_state || last_time step hidden state (*2) , cell_state for last time step
    output,state_h,state_c = tf.keras.layers.LSTM(64,return_state=True)(input)
    #print(output,state_h,state_c)
    output = tf.keras.layers.Dense(32,activation='tanh')(output)
    output = tf.keras.layers.Dense(1)(output)
    strategy = tf.reshape(output,(30,n_sims))
    strategy = tf.cast(strategy,tf.float64)
    option = tf.maximum(STOCK_T - STOCK_K,0)
    Hedging_PnL = - option + tf.reshape(tf.reduce_sum(STOCK_dS * strategy, axis=0),(-1,1))
    Hedging_PnL_paths = - np.reshape(option,(n_sims,)) + STOCK_dS * strategy
    #print(Hedging_PnL_paths) #30,10000
    #CVaR, idx = tf.math.top_k(-Hedging_PnL,k=tf.cast((1-alpha)* n_sims, tf.int64))
    #print(tf.keras.backend.eval(tf.reshape(-Hedging_PnL,(-1,))))
    #CVaR, idx = tf.math.top_k(-Hedging_PnL,k=tf.cast((1-alpha)* n_sims, tf.int64))
    CVaR = tf.reduce_mean(-Hedging_PnL)
    print(Hedging_PnL.shape,CVaR)
    model = tf.keras.Model(inputs=input,outputs=CVaR)
    #model.summary()
    #print('hi',STOCK_dS.shape,strategy.shape,tf.reduce_sum(STOCK_dS * strategy, axis=0),option.shape,STOCK_T.shape,STOCK_K.shape)
    print('stop')
    return model
  

In [ ]:
def deep_lstm_model():
    batch_size = 100
    inputs = STOCK_GRAPH[:batch_size,:,]
    print(inputs.shape)
    lstm = tf.keras.layers.RNN(tf.keras.layers.LSTMCell(4),return_state=True)
    output , state_c, state_h = lstm(inputs)
    #print(output,state_c,state_h)
    output = tf.keras.layers.Dense(32,activation='tanh')(output)
    output = tf.keras.layers.Dense(1)(output)
    output = np.array(output)

    print(output.shape) #(batch_size,1)
    strategy = output
    #strategy = np.reshape(output,(30,-1))
    #print(strategy.shape)
    #strategy = tf.cast(strategy,tf.float64)
    
    option = np.maximum(STOCK_T[:batch_size] - STOCK_K[:100],0)
    print(option.shape) # (100,1)
    print('cal',(STOCK_dS[:batch_size] * strategy).shape)
    print(np.sum(STOCK_dS[:batch_size] * strategy, axis=1).shape) # (100,1)
    Hedging_PnL = -option + np.sum(STOCK_dS[:batch_size] * strategy, axis=1)

    Hedging_PnL_paths = - option + STOCK_dS[:batch_size] * strategy
    find_k = int((1-alpha)*batch_size)
    print(find_k)
    top_k = np.partition(Hedging_PnL, -find_k)[-find_k:]
    print(top_k)

    #print(Hedging_PnL_paths) #30,10000
    CVaR = np.mean(-Hedging_PnL)
    print(Hedging_PnL.shape,CVaR)
    opt = tf.keras.optimizers.Adam(learning_rate=0.1)
    mse = tf.keras.losses.MeanSquaredError()
    var1 = tf.Variable(CVaR)
    loss = lambda: (var1 ** 2)/2.0       # d(loss)/d(var1) == var1
    step_count = opt.minimize(loss, [var1]).numpy()
    # The first step is `-learning_rate*sign(grad)`
    var1.numpy()
deep_lstm_model()


(100, 30, 1)
(100, 1)
(100, 1)
cal (100, 30)
(100,)
50
[[-4.13717274e+00 -4.13717274e+00 -4.13717273e+00 ... -4.13717250e+00
  -4.13717259e+00 -4.13717260e+00]
 [-2.19593873e-08 -1.49921644e-08 -1.37875197e-08 ...  2.24660447e-07
   1.34508942e-07  1.20395253e-07]
 [-2.19593873e-08 -1.49921644e-08 -1.37875197e-08 ...  2.24660447e-07
   1.34508942e-07  1.20395253e-07]
 ...
 [-7.48687502e+00 -7.48687502e+00 -7.48687501e+00 ... -7.48687478e+00
  -7.48687487e+00 -7.48687488e+00]
 [-2.19593873e-08 -1.49921644e-08 -1.37875197e-08 ...  2.24660447e-07
   1.34508942e-07  1.20395253e-07]
 [-2.29363976e+00 -2.29363975e+00 -2.29363975e+00 ... -2.29363951e+00
  -2.29363960e+00 -2.29363961e+00]]
(100, 100) 2.830316510816873


In [ ]:
model = deep_lstm_model()

model.compile(loss='mse',optimizer='adam')
#hist = model.fit(STOCK_GRAPH[:5000,:,],STOCK_T[:5000])



InvalidArgumentError: ignored